In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
import random

from PIL import Image
from google.colab import drive
from collections import Counter

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision import transforms as T
from torch.utils import data

In [2]:
class FashionMnist_Dataset(data.Dataset):
    def __init__(self, data_path, transform, mode):
        self.data_path = data_path
        self.transform = transform
        self.mode = mode
      
        df = pd.read_csv(self.data_path, header=None)
        self.X = df.iloc[:, 1:].values     # 데이터샘플
        self.y = df.iloc[:, 0].values      # 타겟레이블

        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.5, random_state=1, stratify=self.y)
        self.num_data = int(self.x_train.shape[0])

    def __getitem__(self, index):
      if self.mode == 'train':
        self.res = Image.fromarray(self.x_train[index].reshape(28,28).astype('uint8'))
        return self.transform(self.res), self.y_train[index]
      else:
        self.res = Image.fromarray(self.x_test[index].reshape(28,28).astype('uint8'))
        return self.transform(self.res), self.y_test[index]

    def __len__(self):
        return self.num_data


def get_loader(data_path, mode='train', batch_size=50):
    """Build and return a data loader."""
    if mode == 'train':
      transform = T.Compose([
          T.RandomHorizontalFlip(),
          T.RandomCrop(28),
          T.Resize(size=(28, 28), interpolation=3),
          T.ToTensor()
      ])
    else:
      transform = T.Compose([
          T.ToTensor()
      ])
    dataset = FashionMnist_Dataset(data_path, transform, mode)
    data_loader = data.DataLoader(dataset=dataset,
                             batch_size=batch_size,
                             shuffle=(mode=='train'), pin_memory=True, drop_last=(mode=='train'))
    return data_loader

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

drive.mount('/gdrive')
train_data = get_loader('/gdrive/My Drive/ML_전기/텀프/image-10k.csv')
test_data = get_loader('/gdrive/My Drive/ML_전기/텀프/image-10k.csv', mode='test')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [4]:
linear1 = torch.nn.Linear(784, 128, bias=True)
#linear2 = torch.nn.Linear(256, 64, bias=True)
linear2 = torch.nn.Linear(128, 10, bias=True)

# 활성화함수 모듈을 생성

sigmoid = torch.nn.Sigmoid()
relu = torch.nn.ReLU()
#relu = torch.nn.LeakyReLU()

# 만들어진 층, 활성화함수 모듈들을 순서대로 연결해서 모델 생성
# 타겟 연산 플랫폼을 설정

model = torch.nn.Sequential(linear1, sigmoid, linear2)
model.to(device)

criterion = torch.nn.CrossEntropyLoss()  # 내부에 Softmax를 포함함
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [5]:
for epoch in range(150):

    avg_cost = 0
    total_batch = len(train_data)

    # data_loader의 데이터는 앞서 지정한 배치사이즈로 쪼개져 있음
    for X, T in train_data:
        X = X.view(-1, 28 * 28).to(device)
        T = T.to(device)

        optimizer.zero_grad()         # 기존 계산한 경사값 삭제
        output = model(X)             # 순방향 연산
        cost = criterion(output, T)   # 손실함수 설정
        cost.backward()               # 경사값 계산
        optimizer.step()              # 업데이트 1회 수행

        avg_cost += cost / total_batch     # 평균 손실함수값 계산

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
print('Learning finished')

Epoch: 0001 cost = 1.978010535
Epoch: 0002 cost = 1.368873358
Epoch: 0003 cost = 1.103499055
Epoch: 0004 cost = 0.967463911
Epoch: 0005 cost = 0.884060323
Epoch: 0006 cost = 0.824519694
Epoch: 0007 cost = 0.781889141
Epoch: 0008 cost = 0.749092162
Epoch: 0009 cost = 0.721059859
Epoch: 0010 cost = 0.698278129
Epoch: 0011 cost = 0.676137447
Epoch: 0012 cost = 0.659371614
Epoch: 0013 cost = 0.639956355
Epoch: 0014 cost = 0.627257943
Epoch: 0015 cost = 0.615751684
Epoch: 0016 cost = 0.600672424
Epoch: 0017 cost = 0.588854373
Epoch: 0018 cost = 0.580291867
Epoch: 0019 cost = 0.574277222
Epoch: 0020 cost = 0.567612469
Epoch: 0021 cost = 0.558715463
Epoch: 0022 cost = 0.550706029
Epoch: 0023 cost = 0.546077013
Epoch: 0024 cost = 0.538836241
Epoch: 0025 cost = 0.533445120
Epoch: 0026 cost = 0.524849176
Epoch: 0027 cost = 0.521557510
Epoch: 0028 cost = 0.515625536
Epoch: 0029 cost = 0.512324393
Epoch: 0030 cost = 0.504974186
Epoch: 0031 cost = 0.502217770
Epoch: 0032 cost = 0.500139177
Epoch: 0

In [6]:
test_loss = 0
correct = 0

with torch.no_grad():
  for x, y in test_data:
    x = x.view(-1, 28*28).to(device)
    y = y.to(device)

    output = model(x)

    test_loss += F.cross_entropy(output, y, reduction='sum').item()
    pred = output.max(1, keepdim=True)[1]
    correct += pred.eq(y.view_as(pred)).sum().item()

test_loss /= 5000
test_accuracy = 100 * correct / 5000
print(test_loss, test_accuracy)

0.45785281620025636 84.7
